# ANN with Two Layer

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
import nltk
from nltk import word_tokenize, ngrams
from nltk.classify import SklearnClassifier
np.random.seed(25)


In [6]:
# Reading train CSV file directly from Web, and store it in a pandas DataFrame:
train = pd.read_csv('D:/Fauzan/Study PhD/Semester 1/Machine Learning/Homework/HW4/spooky-author-identification/spooky-author-identification/train/train.csv')

# Print first 5 rows
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [7]:
# Reading test CSV file directly from Web, and store it in a pandas DataFrame:
test = pd.read_csv("D:/Fauzan/Study PhD/Semester 1/Machine Learning/Homework/HW4/spooky-author-identification/spooky-author-identification/test/test.csv")

# Print first 5 rows
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [8]:
# Target Mapping
mapping_target = {'EAP':0, 'HPL':1, 'MWS':2}
train = train.replace({'author':mapping_target})

In [9]:
test_id = test['id']
target = train['author']

In [10]:
# function to clean data
import string
import itertools 
import re
from nltk.stem import WordNetLemmatizer
from string import punctuation

def preprocess(text):
    text = text.strip()
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

def cleanData(text, lowercase = False, remove_stops = False, stemming = False, lemmatization = False):
    
    txt = str(text)
    
    txt = re.sub(r'[^A-Za-z\s]',r' ',txt)
    
     
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in txt.split()])
    
    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w, pos='v') for w in txt.split()])

    return txt

In [11]:
train['text'] = train['text'].map(lambda x: preprocess(x))
test['text'] = test['text'].map(lambda x: preprocess(x))

In [12]:
# clean text
train['text'] = train['text'].map(lambda x: cleanData(x, lowercase=True, remove_stops=False, stemming=False, lemmatization = False))
test['text'] = test['text'].map(lambda x: cleanData(x, lowercase=True, remove_stops=False, stemming=False, lemmatization = False))

In [20]:
np.random.seed(25)
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dense
from keras.models import Model
from keras import backend as K
from keras.layers import Convolution1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
import codecs

In [21]:
MAX_SEQUENCE_LENGTH = 256
MAX_NB_WORDS = 200000

In [22]:
def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams

In [23]:
n_gram_max = 2
print('Processing text dataset')
texts_1 = []
for text in train['text']:
    text = text.split()
    texts_1.append(' '.join(add_ngram(text, n_gram_max)))
    

labels = train['author']  # list of label ids

print('Found %s texts.' % len(texts_1))
test_texts_1 = []
for text in test['text']:
    text = text.split()
    test_texts_1.append(' '.join(add_ngram(text, n_gram_max)))
print('Found %s texts.' % len(test_texts_1))

Processing text dataset
Found 19579 texts.
Found 8392 texts.


In [83]:
min_count = 2
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(texts_1)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=True, filters='')
tokenizer.fit_on_texts(texts_1)


sequences_1 = tokenizer.texts_to_sequences(texts_1)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))

test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)

print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
#test_labels = np.array(test_labels)
del test_sequences_1
del sequences_1
import gc
gc.collect()

Shape of data tensor: (19579, 256)
Shape of label tensor: (19579,)


843

In [25]:
nb_words = np.max(data_1) + 1 #min(MAX_NB_WORDS, len(word_index)) + 1

In [75]:
model = Sequential()
# model.add(Conv1D(64, 5, padding='valid', activation='relu', batch_input_shape=(None, MAX_SEQUENCE_LENGTH, 1)))
model.add(Dense(16, activation = 'relu', input_dim = MAX_SEQUENCE_LENGTH))
# model.add(Embedding(nb_words,3,input_length=MAX_SEQUENCE_LENGTH))
# model.add(GlobalAveragePooling1D())
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [76]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 16)                4112      
                                                                 
 dense_20 (Dense)            (None, 3)                 51        
                                                                 
Total params: 4,163
Trainable params: 4,163
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.fit(data_1, to_categorical(labels), validation_split=0.2, epochs=15, batch_size=16)

Epoch 1/15
979/979 [==============================] - 2s 2ms/step - loss: 550.7497 - accuracy: 0.3762 - val_loss: 14.5507 - val_accuracy: 0.3933
Epoch 2/15
979/979 [==============================] - 2s 2ms/step - loss: 5.0365 - accuracy: 0.4011 - val_loss: 3.7502 - val_accuracy: 0.3943
Epoch 3/15
979/979 [==============================] - 2s 2ms/step - loss: 1.7885 - accuracy: 0.4025 - val_loss: 2.3830 - val_accuracy: 0.3950
Epoch 4/15
979/979 [==============================] - 2s 2ms/step - loss: 1.2708 - accuracy: 0.4041 - val_loss: 2.0516 - val_accuracy: 0.3963
Epoch 5/15
979/979 [==============================] - 2s 2ms/step - loss: 1.1960 - accuracy: 0.4044 - val_loss: 1.7899 - val_accuracy: 0.3963
Epoch 6/15
979/979 [==============================] - 2s 2ms/step - loss: 1.1366 - accuracy: 0.4049 - val_loss: 1.4665 - val_accuracy: 0.3981
Epoch 7/15
979/979 [==============================] - 2s 2ms/step - loss: 1.1274 - accuracy: 0.4047 - val_loss: 1.1994 - val_accuracy: 0.3996
Epo

In [51]:
preds = model.predict(test_data_1)

263/263 [==============================] - 0s 1ms/step


In [52]:
result = pd.DataFrame()
result['id'] = test_id
result['EAP'] = [x[0] for x in preds]
result['HPL'] = [x[1] for x in preds]
result['MWS'] = [x[2] for x in preds]

result.to_csv("result.csv", index=False)

In [53]:
result.head()

,id,EAP,HPL,MWS
0,id02310,0.402314,0.289065,0.308621
1,id24541,0.402314,0.289065,0.308621
2,id00134,0.402314,0.289065,0.308621
3,id27757,0.402314,0.289065,0.308621
4,id04081,0.402314,0.289065,0.308621
